# Bussiness Questions

Bussiness Questions (BQ) answered with the generated tables: 


**BQ1.** Average popularity and IMDb rating for each genre found with counts.  
**BQ2.** Top 5 movie actors with the best popularity and rating.  
**BQ3.** Top 5 movie actors with the worst popularity and rating.  
**BQ4.** Top 5 movie directors with the best popularity and rating.  
**BQ5.** Top 5 movie directors with the worst popularity and rating.  
**BQ6.** Top 5 most common pairs of actors and directors, e.g. (Tim burton - Jhony Deep).  
**BQ7.** Top 5 most common pairs of actors, e.g. (Al Pacino - Robert de Niro).  
**BQ8.** Best actors and directors for each genre (use preferred metric and explain the choice).  

In [1]:
# Libraries used
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

In [2]:
# Create a Spark Session
spark = SparkSession.builder.appName('Challenge').getOrCreate()

# Read Parquet tables into Spark Data Frames

### t_movies

In [3]:
# Read Parquet file into Data Frame
t_movies = spark.read.option('header','true').parquet('master/t_movies.parquet')

# Manually define Schema
t_movies = t_movies.select(
    t_movies.movie_id.cast('string'),
    t_movies.title.cast('string'),
    t_movies.rating.cast('double'),
    t_movies.popularity.cast('int')
)

t_movies.printSchema()
t_movies.show(5)

root
 |-- movie_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- popularity: integer (nullable = true)

+--------+--------------------+------+----------+
|movie_id|               title|rating|popularity|
+--------+--------------------+------+----------+
|       1|The Shawshank Red...|   9.3|        71|
|       2|       The Godfather|   9.2|        68|
|       3|     The Dark Knight|   9.0|       107|
|       4|The Godfather Par...|   9.0|       245|
|       5|        12 Angry Men|   9.0|       276|
+--------+--------------------+------+----------+
only showing top 5 rows



### t_genres

In [4]:
# t_genres
t_genres = spark.read.parquet('master/t_genres.parquet', header=True)

t_genres.printSchema()
t_genres.show(5)

root
 |-- movie_id: long (nullable = true)
 |-- genre: string (nullable = true)

+--------+------+
|movie_id| genre|
+--------+------+
|       1| Drama|
|       2| Crime|
|       2| Drama|
|       3|Action|
|       3| Crime|
+--------+------+
only showing top 5 rows



### t_actors

In [5]:
# t_actors
t_actors = spark.read.parquet('master/t_actors.parquet', header=True)

t_actors.printSchema()
t_actors.show(5)

root
 |-- movie_id: long (nullable = true)
 |-- actor: string (nullable = true)

+--------+--------------+
|movie_id|         actor|
+--------+--------------+
|       1|   Tim Robbins|
|       1|Morgan Freeman|
|       1|    Bob Gunton|
|       1|William Sadler|
|       1|  Clancy Brown|
+--------+--------------+
only showing top 5 rows



### t_directors

In [6]:
# t_directors
t_directors = spark.read.parquet('master/t_directors.parquet', header=True)

t_directors.printSchema()
t_directors.show(5)

root
 |-- movie_id: long (nullable = true)
 |-- director: string (nullable = true)

+--------+--------------------+
|movie_id|            director|
+--------+--------------------+
|       1|      Frank Darabont|
|       2|Francis Ford Coppola|
|       3|   Christopher Nolan|
|       4|Francis Ford Coppola|
|       5|        Sidney Lumet|
+--------+--------------------+
only showing top 5 rows



## Create temporary views to run SQL queries.

In [7]:
# Create views of the tables to run SQL queries
t_movies.createOrReplaceTempView('t_movies')  
t_genres.createOrReplaceTempView('t_genres')  
t_actors.createOrReplaceTempView('t_actors')  
t_directors.createOrReplaceTempView('t_directors')

## Join t_movies & t_genres.

In [8]:
join1 = spark.sql("""
    SELECT m.movie_id, m.title, g.genre, m.rating, m.popularity 
    FROM t_movies m
    INNER JOIN t_genres g
    ON m.movie_id = g.movie_id
""")

# Display
print(f'\n{join1.count()} rows in total')
join1.show(5)


626 rows in total
+--------+--------------------+-----+------+----------+
|movie_id|               title|genre|rating|popularity|
+--------+--------------------+-----+------+----------+
|       1|The Shawshank Red...|Drama|   9.3|        71|
|       2|       The Godfather|Drama|   9.2|        68|
|       2|       The Godfather|Crime|   9.2|        68|
|       3|     The Dark Knight|Drama|   9.0|       107|
|       3|     The Dark Knight|Crime|   9.0|       107|
+--------+--------------------+-----+------+----------+
only showing top 5 rows



In [9]:
# Create temporary view of join1 to run queries
join1.createOrReplaceTempView('join1')  

**Count and visualize total number of genres.**

In [10]:
genre_distinct = spark.sql("""
    SELECT DISTINCT genre
    FROM t_genres
    ORDER BY genre
""")

genre_count = genre_distinct.count()

# Display
print(f'\n{genre_count} distinct values')
genre_distinct.show(genre_count)


21 distinct values
+---------+
|    genre|
+---------+
|   Action|
|Adventure|
|Animation|
|Biography|
|   Comedy|
|    Crime|
|    Drama|
|   Family|
|  Fantasy|
|Film-Noir|
|  History|
|   Horror|
|    Music|
|  Musical|
|  Mystery|
|  Romance|
|   Sci-Fi|
|    Sport|
| Thriller|
|      War|
|  Western|
+---------+



### BQ1. Average popularity and IMDb rating for each genre found with counts.

In [11]:
# Average rating by genre
spark.sql("""
    SELECT
        genre,
        CAST(AVG(rating) AS DECIMAL(8,6)) AS avg_rating,
        COUNT(genre) AS genre_count
    FROM join1
    GROUP BY genre
    ORDER BY avg_rating DESC
""").show(genre_count)

+---------+----------+-----------+
|    genre|avg_rating|genre_count|
+---------+----------+-----------+
|    Music|  8.400000|          4|
|    Crime|  8.360784|         51|
|   Sci-Fi|  8.360000|         20|
|   Action|  8.348000|         50|
|   Horror|  8.340000|          5|
|  Western|  8.328571|          7|
|  Mystery|  8.325806|         31|
|    Drama|  8.320339|        177|
|  Fantasy|  8.307143|         14|
|  Musical|  8.300000|          1|
|Adventure|  8.296667|         60|
|  Romance|  8.295652|         23|
|      War|  8.291304|         23|
|  History|  8.270000|         10|
| Thriller|  8.261290|         31|
|   Family|  8.253846|         13|
|Biography|  8.248276|         29|
|Animation|  8.247826|         23|
|   Comedy|  8.228889|         45|
|Film-Noir|  8.225000|          4|
|    Sport|  8.160000|          5|
+---------+----------+-----------+



In [12]:
# Average rating by genre
spark.sql("""
    SELECT
        genre,
        CAST(AVG(popularity) AS DECIMAL(7,2)) AS avg_popularity,
        COUNT(genre) AS genre_count
    FROM join1
    GROUP BY genre
    ORDER BY avg_popularity
""").show(genre_count)

+---------+--------------+-----------+
|    genre|avg_popularity|genre_count|
+---------+--------------+-----------+
|   Sci-Fi|        415.35|         20|
|   Horror|        442.40|          5|
|    Music|        535.50|          4|
|   Action|        726.52|         50|
|Adventure|        777.55|         60|
|  Fantasy|        795.57|         14|
|    Sport|        817.60|          5|
|Biography|        945.69|         29|
|    Crime|        990.84|         51|
| Thriller|       1054.26|         31|
|  History|       1086.10|         10|
|    Drama|       1090.73|        177|
|   Comedy|       1172.40|         45|
|   Family|       1175.38|         13|
|  Romance|       1223.22|         23|
|Animation|       1256.52|         23|
|  Mystery|       1272.52|         31|
|  Western|       1418.29|          7|
|      War|       1446.26|         23|
|  Musical|       1775.00|          1|
|Film-Noir|       2262.50|          4|
+---------+--------------+-----------+



## Join t_movies & t_actors.

In [13]:
join2 = spark.sql("""
    SELECT m.movie_id, m.title, a.actor, m.rating, m.popularity
    FROM t_movies m
    INNER JOIN t_actors a
    ON m.movie_id = a.movie_id
""")

# Display
print(f'\n{join2.count()} rows in total')
join2.show(5)


4360 rows in total
+--------+--------------------+--------------+------+----------+
|movie_id|               title|         actor|rating|popularity|
+--------+--------------------+--------------+------+----------+
|       1|The Shawshank Red...|   Tim Robbins|   9.3|        71|
|       1|The Shawshank Red...|Morgan Freeman|   9.3|        71|
|       1|The Shawshank Red...|    Bob Gunton|   9.3|        71|
|       1|The Shawshank Red...|William Sadler|   9.3|        71|
|       1|The Shawshank Red...|  Clancy Brown|   9.3|        71|
+--------+--------------------+--------------+------+----------+
only showing top 5 rows



In [14]:
# Create temporary view of join2 to run queries
join2.createOrReplaceTempView('join2')  

## BQ2. Top 5 movie actors with the best popularity and rating.

In [15]:
# Actors with best rating (average)
spark.sql("""
    SELECT actor, CAST(AVG(rating) AS DECIMAL(8,6)) AS avg_rating
    FROM join2
    GROUP BY actor
    ORDER BY avg_rating DESC
""").show(5)

+---------------+----------+
|          actor|avg_rating|
+---------------+----------+
|    Tim Robbins|  9.300000|
|   David Proval|  9.300000|
|   Paul McCrane|  9.300000|
|Jude Ciccolella|  9.300000|
|   Joseph Ragno|  9.300000|
+---------------+----------+
only showing top 5 rows



In [16]:
# Actors with best popularity (average)
spark.sql("""
    SELECT actor, CAST(AVG(popularity) AS DECIMAL(7,2)) AS avg_popularity
    FROM join2
    GROUP BY actor
    ORDER BY avg_popularity
""").show(5)

+-------------------+--------------+
|              actor|avg_popularity|
+-------------------+--------------+
|      Lewis Pullman|         25.00|
|     Michele Sakara|         25.00|
|      Carlo Jachino|         25.00|
|Lamberto Maggiorani|         25.00|
|          Jay Ellis|         25.00|
+-------------------+--------------+
only showing top 5 rows



### BQ2 Alternative approach.

In [17]:
# Actors with best rating (cumulative)
spark.sql("""
    SELECT actor, CAST(SUM(rating) AS DECIMAL(5,2)) AS tot_rating
    FROM join2
    GROUP BY actor
    ORDER BY tot_rating DESC
""").show(5)

+-----------------+----------+
|            actor|tot_rating|
+-----------------+----------+
|   Robert De Niro|     75.40|
|   Morgan Freeman|     59.80|
|    Harrison Ford|     58.80|
|John Ratzenberger|     57.60|
|    Michael Caine|     51.50|
+-----------------+----------+
only showing top 5 rows



In [18]:
# Actors with best popularity (average popularity & at least 3 movies in the Top-250)
spark.sql("""
    SELECT actor, CAST(AVG(popularity) AS DECIMAL(7,2)) AS avg_popularity, COUNT(actor) AS movies
    FROM join2
    GROUP BY actor
    HAVING movies >= 3
    ORDER BY avg_popularity
""").show(5)

+--------------------+--------------+------+
|               actor|avg_popularity|movies|
+--------------------+--------------+------+
|         Talia Shire|        139.67|     3|
|          Matt Damon|        191.75|     4|
|   Leonardo DiCaprio|        203.17|     6|
|Benedict Cumberbatch|        207.00|     3|
|         Tom Holland|        207.00|     3|
+--------------------+--------------+------+
only showing top 5 rows



## BQ3. Top 5 movie actors with the worst popularity and rating.

In [19]:
# Actors with worst rating (average)
spark.sql("""
    SELECT actor, CAST(AVG(rating) AS DECIMAL(8,6)) AS avg_rating
    FROM join2
    GROUP BY actor
    ORDER BY avg_rating
""").show(5)

+--------------------+----------+
|               actor|avg_rating|
+--------------------+----------+
|         Tom Everett|  8.000000|
|Nathan Lee Chasin...|  8.000000|
|     Craig T. Nelson|  8.000000|
|    Jonathan Freeman|  8.000000|
|     Dominique Louis|  8.000000|
+--------------------+----------+
only showing top 5 rows



In [20]:
# Actors with worst popularity (average)
spark.sql("""
    SELECT actor, CAST(AVG(popularity) AS DECIMAL(7,2)) AS avg_popularity
    FROM join2
    GROUP BY actor
    ORDER BY avg_popularity DESC
""").show(5)

+-------------------+--------------+
|              actor|avg_popularity|
+-------------------+--------------+
| Aparshakti Khurana|       4959.00|
|      Mahesh Balraj|       4959.00|
|       Anmol Charan|       4959.00|
| Fatima Sana Shaikh|       4959.00|
|Karamveer Choudhary|       4959.00|
+-------------------+--------------+
only showing top 5 rows



## Join t_movies & t_directors.

In [21]:
join3 = spark.sql("""
    SELECT m.movie_id, m.title, d.director, m.rating, m.popularity 
    FROM t_movies m
    INNER JOIN t_directors d
    ON m.movie_id = d.movie_id
""")

# Display
print(f'\n{join3.count()} rows in total')
join3.show(5)


277 rows in total
+--------+--------------------+--------------------+------+----------+
|movie_id|               title|            director|rating|popularity|
+--------+--------------------+--------------------+------+----------+
|       1|The Shawshank Red...|      Frank Darabont|   9.3|        71|
|       2|       The Godfather|Francis Ford Coppola|   9.2|        68|
|       3|     The Dark Knight|   Christopher Nolan|   9.0|       107|
|       4|The Godfather Par...|Francis Ford Coppola|   9.0|       245|
|       5|        12 Angry Men|        Sidney Lumet|   9.0|       276|
+--------+--------------------+--------------------+------+----------+
only showing top 5 rows



In [22]:
# Create temporary view of join3 to run queries
join3.createOrReplaceTempView('join3')

## BQ4. Top 5 movie directors with the best popularity and rating.

In [23]:
# Directors with best rating (average)
spark.sql("""
    SELECT director, CAST(AVG(rating) AS DECIMAL(8,6)) AS avg_rating
    FROM join3
    GROUP BY director
    ORDER BY avg_rating DESC
""").show(5)

+--------------------+----------+
|            director|avg_rating|
+--------------------+----------+
|      Frank Darabont|  8.950000|
|Francis Ford Coppola|  8.900000|
|       Peter Jackson|  8.866667|
|       T.J. Gnanavel|  8.800000|
|      Lana Wachowski|  8.700000|
+--------------------+----------+
only showing top 5 rows



In [24]:
# Actors with best popularity (average)
spark.sql("""
    SELECT director, CAST(AVG(popularity) AS DECIMAL(7,2)) AS avg_popularity
    FROM join3
    GROUP BY director
    ORDER BY avg_popularity
""").show(5)

+----------------+--------------+
|        director|avg_popularity|
+----------------+--------------+
|Vittorio De Sica|         25.00|
| Joseph Kosinski|         25.00|
|     Tate Taylor|         97.00|
|John G. Avildsen|        106.00|
|    Gus Van Sant|        124.00|
+----------------+--------------+
only showing top 5 rows



### BQ4 Alternative approach.

In [25]:
# Directors with best rating (cumulative)
spark.sql("""
    SELECT director, CAST(SUM(rating) AS DECIMAL(5,2)) AS tot_rating
    FROM join3
    GROUP BY director
    ORDER BY tot_rating DESC
""").show(5)

+-----------------+----------+
|         director|tot_rating|
+-----------------+----------+
|Christopher Nolan|     59.90|
| Steven Spielberg|     58.60|
|  Stanley Kubrick|     58.20|
|  Martin Scorsese|     58.20|
|   Akira Kurosawa|     58.10|
+-----------------+----------+
only showing top 5 rows



In [26]:
# Directors with best popularity (average popularity & at least 4 movies in the Top-250)
spark.sql("""
    SELECT director, CAST(AVG(popularity) AS DECIMAL(7,2)) AS avg_popularity, COUNT(director) AS movies
    FROM join3
    GROUP BY director
    HAVING movies >= 4
    ORDER BY avg_popularity
""").show(5)

+-----------------+--------------+------+
|         director|avg_popularity|movies|
+-----------------+--------------+------+
| Steven Spielberg|        263.43|     7|
|Christopher Nolan|        270.86|     7|
|Quentin Tarantino|        296.20|     5|
|  Martin Scorsese|        375.43|     7|
|      Lee Unkrich|       1000.50|     4|
+-----------------+--------------+------+
only showing top 5 rows



## BQ5. Top 5 movie directors with the worst popularity and rating.

In [27]:
# Directors with worst rating (average)
spark.sql("""
    SELECT director, CAST(AVG(rating) AS DECIMAL(8,6)) AS avg_rating
    FROM join3
    GROUP BY director
    ORDER BY avg_rating
""").show(5)

+--------------------+----------+
|            director|avg_rating|
+--------------------+----------+
|Richard Attenborough|  8.000000|
|         John Musker|  8.000000|
|        Ron Clements|  8.000000|
|       Kevin Costner|  8.000000|
|           Brad Bird|  8.066667|
+--------------------+----------+
only showing top 5 rows



In [28]:
# Directors with worst popularity (average)
spark.sql("""
    SELECT director, CAST(AVG(popularity) AS DECIMAL(7,2)) AS avg_popularity
    FROM join3
    GROUP BY director
    ORDER BY avg_popularity DESC
""").show(5)

+-----------------+--------------+
|         director|avg_popularity|
+-----------------+--------------+
|    Nitesh Tiwari|       4959.00|
|      Adam Elliot|       4598.00|
|François Truffaut|       4428.00|
|    T.J. Gnanavel|       4386.00|
|      John Huston|       4044.00|
+-----------------+--------------+
only showing top 5 rows



## Join t_directors & t_actors.

In [29]:
join4 = spark.sql("""
    SELECT d.movie_id, d.director, a.actor
    FROM t_directors d
    INNER JOIN t_actors a
    ON d.movie_id = a.movie_id
""")

# Display
print(f'\n{join4.count()} rows in total')
join4.show(5)


4833 rows in total
+--------+--------------+--------------+
|movie_id|      director|         actor|
+--------+--------------+--------------+
|       1|Frank Darabont|   Tim Robbins|
|       1|Frank Darabont|Morgan Freeman|
|       1|Frank Darabont|    Bob Gunton|
|       1|Frank Darabont|William Sadler|
|       1|Frank Darabont|  Clancy Brown|
+--------+--------------+--------------+
only showing top 5 rows



In [30]:
# Create temporary view of join4 to run queries
join4.createOrReplaceTempView('join4')  

## BQ6. Top 5 most common pairs of actors and directors. 
**e.g. (Tim burton - Jhony Deep)**

In [31]:
spark.sql("""
SELECT director, actor, COUNT(*) AS count 
FROM join4
GROUP BY director, actor
ORDER BY count DESC
""").show(5)

+-----------------+---------------+-----+
|         director|          actor|count|
+-----------------+---------------+-----+
|Christopher Nolan|  Michael Caine|    6|
|  Charles Chaplin|Charles Chaplin|    5|
|   Akira Kurosawa|Takashi Shimura|    5|
|Christopher Nolan| Christian Bale|    4|
|  Charles Chaplin|  Henry Bergman|    4|
+-----------------+---------------+-----+
only showing top 5 rows



## BQ7. Top 5 most common pairs of actors.
**e.g. (Al Pacino - Robert de Niro)**

In [32]:
spark.sql("""
    SELECT a1.actor AS actor1, a2.actor AS actor2, COUNT(*) AS count
    FROM t_actors a1
    INNER JOIN t_actors a2 
    ON a1.movie_id = a2.movie_id
    WHERE a1.actor < a2.actor
    GROUP BY a1.actor, a2.actor
    ORDER BY count DESC
""").show(5)

+---------------+---------------+-----+
|         actor1|         actor2|count|
+---------------+---------------+-----+
| Christian Bale|  Michael Caine|    4|
|Charles Chaplin|  Henry Bergman|    4|
|Takashi Shimura| Toshirô Mifune|    4|
|      Joe Pesci| Robert De Niro|    4|
|  Minoru Chiaki|Takashi Shimura|    4|
+---------------+---------------+-----+
only showing top 5 rows



## Join t_movies, t_genres, t_actors.

In [33]:
join5 = spark.sql("""
    SELECT m.movie_id, m.title, g.genre, a.actor, m.rating, m.popularity
    FROM t_movies m
    INNER JOIN t_genres g
    ON m.movie_id = g.movie_id
    INNER JOIN t_actors a
    ON m.movie_id = a.movie_id
""")

# Display
print(f'\n{join5.count()} rows in total')
join5.show(5)


10939 rows in total
+--------+--------------------+-----+---------------+------+----------+
|movie_id|               title|genre|          actor|rating|popularity|
+--------+--------------------+-----+---------------+------+----------+
|       1|The Shawshank Red...|Drama|     Scott Mann|   9.3|        71|
|       1|The Shawshank Red...|Drama|   Renee Blaine|   9.3|        71|
|       1|The Shawshank Red...|Drama|   Paul McCrane|   9.3|        71|
|       1|The Shawshank Red...|Drama|Jude Ciccolella|   9.3|        71|
|       1|The Shawshank Red...|Drama|   Joseph Ragno|   9.3|        71|
+--------+--------------------+-----+---------------+------+----------+
only showing top 5 rows



In [34]:
# Create temporary view of join5 to run queries
join5.createOrReplaceTempView('join5')  

## BQ8. [Part 1] Best actors for each genre.
**use preferred metric and explain the choice

In [35]:
# Actors' rating (cumulative) divided by genre
aux1 = spark.sql("""
    SELECT genre, actor, CAST(SUM(rating) AS DECIMAL(5,2)) AS tot_rating_by_genre
    FROM join5
    GROUP BY genre, actor
    ORDER BY tot_rating_by_genre DESC
""")

# Display
print(f'\n{aux1.count()} rows in total')
aux1.show(5)


10102 rows in total
+---------+-----------------+-------------------+
|    genre|            actor|tot_rating_by_genre|
+---------+-----------------+-------------------+
|    Drama|   Robert De Niro|              75.40|
|    Drama|   Morgan Freeman|              59.80|
|    Crime|   Robert De Niro|              59.10|
|Adventure|John Ratzenberger|              57.60|
|Animation|John Ratzenberger|              57.60|
+---------+-----------------+-------------------+
only showing top 5 rows



In [36]:
# Create temporary view of aux to run queries
aux1.createOrReplaceTempView('aux1')  

In [37]:
# Best rating (cumulative) by genre
spark.sql("""
    SELECT genre, MAX(tot_rating_by_genre) AS max_tot_rating_by_genre
    FROM aux1
    GROUP BY genre
    ORDER BY max_tot_rating_by_genre DESC
""").show(genre_count)

+---------+-----------------------+
|    genre|max_tot_rating_by_genre|
+---------+-----------------------+
|    Drama|                  75.40|
|    Crime|                  59.10|
|Adventure|                  57.60|
|Animation|                  57.60|
|   Action|                  50.30|
|   Comedy|                  49.20|
|   Sci-Fi|                  25.90|
|  Fantasy|                  25.60|
|  Mystery|                  25.30|
|  Western|                  25.20|
|Biography|                  17.00|
|  Romance|                  17.00|
|  History|                  17.00|
|   Family|                  16.80|
| Thriller|                  16.80|
|      War|                  16.60|
|   Horror|                   8.50|
|    Music|                   8.50|
|Film-Noir|                   8.40|
|  Musical|                   8.30|
|    Sport|                   8.20|
+---------+-----------------------+



### [Preliminary] Best actors by rating (cumulative) for each genre.

In [38]:
# Best actors by rating (cumulative) for each genre
aux2 = spark.sql("""
    SELECT t3.genre1, t3.actor, t3.tot_rating_by_genre, t3.max_tot_rating_by_genre
    FROM
    (SELECT *
    FROM 
        (SELECT join5.genre AS genre1, actor, CAST(SUM(rating) AS DECIMAL(5,2)) AS tot_rating_by_genre
        FROM join5
        GROUP BY genre, actor
        ORDER BY tot_rating_by_genre DESC) t1
    INNER JOIN
        (SELECT aux1.genre AS genre2, MAX(tot_rating_by_genre) AS max_tot_rating_by_genre
        FROM aux1
        GROUP BY genre) t2
    ON t1.genre1 = t2.genre2) t3
    WHERE t3.tot_rating_by_genre = t3.max_tot_rating_by_genre
    ORDER BY t3.genre1, t3.tot_rating_by_genre DESC
""")

# Display
print(f'\n{aux2.count()} rows in total')
aux2.show(5)


182 rows in total
+---------+-----------------+-------------------+-----------------------+
|   genre1|            actor|tot_rating_by_genre|max_tot_rating_by_genre|
+---------+-----------------+-------------------+-----------------------+
|   Action|    Harrison Ford|              50.30|                  50.30|
|Adventure|John Ratzenberger|              57.60|                  57.60|
|Animation|John Ratzenberger|              57.60|                  57.60|
|Biography|     Ben Kingsley|              17.00|                  17.00|
|   Comedy|John Ratzenberger|              49.20|                  49.20|
+---------+-----------------+-------------------+-----------------------+
only showing top 5 rows



In [39]:
# Create # Create temporary view of aux to run queries
aux2.createOrReplaceTempView('aux2')

***We can see there are several ties (182) with the current metric, thus, we will try to incorporate the popularity score into the mix.***

Let us define the following custom metric M:  

$$M = \frac{(\text{total rating})*(3975 - \text{average popularity})}{1975} $$

In [40]:
# Custom metric for actors by genre
aux3 = spark.sql("""
    SELECT 
        join5.genre AS genre1, 
        actor,
        CAST( SUM(rating) * (-AVG(popularity) + 3975) / 1975 AS DECIMAL(5,2)) AS metric,
        CAST(SUM(rating) AS DECIMAL(5,2)) AS tot_rating_by_genre, 
        CAST(AVG(popularity) AS DECIMAL(7,2)) AS avg_popularity
    FROM join5
    GROUP BY genre1, actor
    ORDER BY genre1, metric DESC
""")

# Display
print(f'\n{aux3.count()} rows in total')
aux3.show(5)


10102 rows in total
+------+--------------+------+-------------------+--------------+
|genre1|         actor|metric|tot_rating_by_genre|avg_popularity|
+------+--------------+------+-------------------+--------------+
|Action| Harrison Ford| 89.28|              50.30|        469.50|
|Action| Orlando Bloom| 65.89|              34.70|        224.50|
|Action| Michael Caine| 64.40|              34.40|        277.75|
|Action|   Gary Oldman| 63.21|              34.10|        314.00|
|Action|Christian Bale| 62.40|              33.70|        318.00|
+------+--------------+------+-------------------+--------------+
only showing top 5 rows



In [41]:
# Create temporary view
aux3.createOrReplaceTempView('aux3')  

### [Preliminary] Best actors for each genre.

In [42]:
# Best actors for each genre - multiple ties!
aux4 = spark.sql("""
    SELECT st1.genre2 AS genre3, aux3.actor, st1.max_metric
    FROM
    (SELECT genre1 AS genre2, MAX(metric) AS max_metric
    FROM aux3
    GROUP BY genre2) st1
    INNER JOIN aux3
    ON st1.genre2 = aux3.genre1 
    WHERE st1.max_metric = aux3.metric
    ORDER BY st1.genre2, aux3.metric DESC
""")

# Display
print(f'\n{aux4.count()} rows in total')
aux4.show(5)


120 rows in total
+---------+-----------------+----------+
|   genre3|            actor|max_metric|
+---------+-----------------+----------+
|   Action|    Harrison Ford|     89.28|
|Adventure|John Ratzenberger|     89.35|
|Animation|John Ratzenberger|     89.35|
|Biography|       P.J. Byrne|     31.80|
|   Comedy|John Ratzenberger|     76.15|
+---------+-----------------+----------+
only showing top 5 rows



In [43]:
# Create temporary view
aux4.createOrReplaceTempView('aux4')  

### The data is not big enough to separate the actors' statistics for some of the genres!  
**Therefore, we settle ties in such categories sorting by inverse alphabetical order and picking one representative.**

## BQ8. [Solution 1]

In [44]:
# Solution [Part 1] - Best actor for each genre
spark.sql("""
    SELECT genre3, MAX(actor) as best_actor
    FROM aux4
    GROUP BY genre3
""").show(genre_count)

+---------+------------------+
|   genre3|        best_actor|
+---------+------------------+
|   Action|     Harrison Ford|
|Adventure| John Ratzenberger|
|Animation| John Ratzenberger|
|Biography|        P.J. Byrne|
|   Comedy| John Ratzenberger|
|    Crime|    Robert De Niro|
|    Drama|    Robert De Niro|
|   Family|         Yô Ôizumi|
|  Fantasy|      Peter Mayhew|
|Film-Noir|Wilfrid Hyde-White|
|  History|      Ben Kingsley|
|   Horror|      Yaphet Kotto|
|    Music|     Tyler Kimball|
|  Musical|         Sue Allen|
|  Mystery|      Kevin Spacey|
|  Romance|       Clark Gable|
|   Sci-Fi|     Michael Caine|
|    Sport|      Thayer David|
| Thriller|      Brett Cullen|
|      War|        Vin Diesel|
|  Western|    Clint Eastwood|
+---------+------------------+



## BQ8. [Part 2] Best directors for each genre.

In [45]:
join6 = spark.sql("""
    SELECT m.movie_id, m.title, g.genre, d.director, m.rating, m.popularity
    FROM t_movies m
    INNER JOIN t_genres g
    ON m.movie_id = g.movie_id
    INNER JOIN t_directors d
    ON m.movie_id = d.movie_id
""")

# Display
print(f'\n{join6.count()} rows in total')
join6.show(5)


702 rows in total
+--------+--------------------+-----+--------------------+------+----------+
|movie_id|               title|genre|            director|rating|popularity|
+--------+--------------------+-----+--------------------+------+----------+
|       1|The Shawshank Red...|Drama|      Frank Darabont|   9.3|        71|
|       2|       The Godfather|Drama|Francis Ford Coppola|   9.2|        68|
|       2|       The Godfather|Crime|Francis Ford Coppola|   9.2|        68|
|       3|     The Dark Knight|Drama|   Christopher Nolan|   9.0|       107|
|       3|     The Dark Knight|Crime|   Christopher Nolan|   9.0|       107|
+--------+--------------------+-----+--------------------+------+----------+
only showing top 5 rows



In [46]:
# Create temporary view 
join6.createOrReplaceTempView('join6')  

In [47]:
join7 = spark.sql("""
    SELECT *
    FROM t_movies m
    INNER JOIN t_genres g
    ON m.movie_id = g.movie_id
    INNER JOIN t_directors d
    ON m.movie_id = d.movie_id
""")

# Display
print(f'\n{join7.count()} rows in total')
join7.show(5)


702 rows in total
+--------+--------------------+------+----------+--------+-----+--------+--------------------+
|movie_id|               title|rating|popularity|movie_id|genre|movie_id|            director|
+--------+--------------------+------+----------+--------+-----+--------+--------------------+
|       1|The Shawshank Red...|   9.3|        71|       1|Drama|       1|      Frank Darabont|
|       2|       The Godfather|   9.2|        68|       2|Drama|       2|Francis Ford Coppola|
|       2|       The Godfather|   9.2|        68|       2|Crime|       2|Francis Ford Coppola|
|       3|     The Dark Knight|   9.0|       107|       3|Drama|       3|   Christopher Nolan|
|       3|     The Dark Knight|   9.0|       107|       3|Crime|       3|   Christopher Nolan|
+--------+--------------------+------+----------+--------+-----+--------+--------------------+
only showing top 5 rows



In [48]:
# Create temporary view
join7.createOrReplaceTempView('join7')  

In [49]:
# Director' rating (cumulative) by genre
auxx1 = spark.sql("""
    SELECT genre, director, CAST(SUM(rating) AS DECIMAL(5,2)) AS tot_rating_by_genre
    FROM join7
    GROUP BY genre, director
    ORDER BY tot_rating_by_genre DESC
""")

# Display
print(f'\n{auxx1.count()} rows in total')
auxx1.show(5)


529 rows in total
+-----+-----------------+-------------------+
|genre|         director|tot_rating_by_genre|
+-----+-----------------+-------------------+
|Drama|   Akira Kurosawa|              58.10|
|Drama|Christopher Nolan|              42.70|
|Crime|  Martin Scorsese|              41.80|
|Drama|  Charles Chaplin|              41.80|
|Drama|  Martin Scorsese|              41.80|
+-----+-----------------+-------------------+
only showing top 5 rows



In [50]:
# Create temporary view
auxx1.createOrReplaceTempView('auxx1')  

In [51]:
# Best rating (cumulative) by genre
spark.sql("""
    SELECT genre, MAX(tot_rating_by_genre) AS max_tot_rating_by_genre
    FROM auxx1
    GROUP BY genre
    ORDER BY max_tot_rating_by_genre DESC
""").show(genre_count)

+---------+-----------------------+
|    genre|max_tot_rating_by_genre|
+---------+-----------------------+
|    Drama|                  58.10|
|   Comedy|                  41.80|
|    Crime|                  41.80|
|  Mystery|                  41.70|
|   Action|                  34.40|
| Thriller|                  33.50|
|      War|                  33.20|
|Animation|                  33.20|
|Adventure|                  33.00|
|   Sci-Fi|                  25.90|
|  Western|                  25.50|
|Biography|                  25.10|
|   Family|                  24.90|
|  Romance|                  17.00|
|Film-Noir|                  16.70|
|  History|                  16.50|
|  Fantasy|                   8.70|
|    Music|                   8.50|
|   Horror|                   8.50|
|  Musical|                   8.30|
|    Sport|                   8.20|
+---------+-----------------------+



### [Preliminary] Best directors by rating (cumulative) for each genre.

In [52]:
# Best directors by rating (cumulative) for each genre
auxx2 = spark.sql("""
    SELECT t3.genre1, t3.director, t3.tot_rating_by_genre, t3.max_tot_rating_by_genre
    FROM
    (SELECT *
    FROM 
        (SELECT join6.genre AS genre1, director, CAST(SUM(rating) AS DECIMAL(5,2)) AS tot_rating_by_genre
        FROM join6
        GROUP BY genre, director
        ORDER BY tot_rating_by_genre DESC) t1
    INNER JOIN
        (SELECT auxx1.genre AS genre2, MAX(tot_rating_by_genre) AS max_tot_rating_by_genre
        FROM auxx1
        GROUP BY genre) t2
    ON t1.genre1 = t2.genre2) t3
    WHERE t3.tot_rating_by_genre = t3.max_tot_rating_by_genre
    ORDER BY t3.genre1, t3.tot_rating_by_genre DESC
""")

# Display
print(f'\n{auxx2.count()} rows in total')
auxx2.show(26)


26 rows in total
+---------+-----------------+-------------------+-----------------------+
|   genre1|         director|tot_rating_by_genre|max_tot_rating_by_genre|
+---------+-----------------+-------------------+-----------------------+
|   Action|Christopher Nolan|              34.40|                  34.40|
|Adventure|      Lee Unkrich|              33.00|                  33.00|
|Animation|   Hayao Miyazaki|              33.20|                  33.20|
|Biography|  Martin Scorsese|              25.10|                  25.10|
|   Comedy|  Charles Chaplin|              41.80|                  41.80|
|    Crime|  Martin Scorsese|              41.80|                  41.80|
|    Drama|   Akira Kurosawa|              58.10|                  58.10|
|   Family|   Hayao Miyazaki|              24.90|                  24.90|
|  Fantasy|   Irvin Kershner|               8.70|                   8.70|
|Film-Noir|     Billy Wilder|              16.70|                  16.70|
|  History|       Me

In [53]:
# Create # Create temporary view of aux to run queries
auxx2.createOrReplaceTempView('auxx2')

***We can see some ties (26) with the current metric, although it is significantly better than the case of the actors. In any case, we can still incorporate the popularity score into the mix by using our custom metric.***

Recall that we defined our custom metric M as:  

$$M = \frac{(\text{total rating})*(3975 - \text{average popularity})}{1975} $$

In [54]:
# Custom metric for directors by genre
auxx3 = spark.sql("""
    SELECT 
        join6.genre AS genre1, 
        director,
        CAST( SUM(rating) * (-AVG(popularity) + 3975) / 1975 AS DECIMAL(5,2)) AS metric,
        CAST(SUM(rating) AS DECIMAL(5,2)) AS tot_rating_by_genre, 
        CAST(AVG(popularity) AS DECIMAL(7,2)) AS avg_popularity
    FROM join6
    GROUP BY genre1, director
    ORDER BY genre1, metric DESC
""")

# Display
print(f'\n{auxx3.count()} rows in total')
auxx3.show(5)


529 rows in total
+------+-----------------+------+-------------------+--------------+
|genre1|         director|metric|tot_rating_by_genre|avg_popularity|
+------+-----------------+------+-------------------+--------------+
|Action|Christopher Nolan| 64.40|              34.40|        277.75|
|Action|    Peter Jackson| 50.56|              26.60|        221.33|
|Action| Steven Spielberg| 47.25|              24.80|        212.33|
|Action|    James Cameron| 45.02|              25.10|        432.33|
|Action|   Akira Kurosawa| 33.15|              25.00|       1356.33|
+------+-----------------+------+-------------------+--------------+
only showing top 5 rows



In [55]:
# Create temporary view
auxx3.createOrReplaceTempView('auxx3')  

## BQ8. [Solution 2]

In [56]:
# Best directors for each genre - only one tie!
auxx4 = spark.sql("""
    SELECT st1.genre2 AS genre3, auxx3.director, st1.max_metric
    FROM
    (SELECT genre1 AS genre2, MAX(metric) AS max_metric
    FROM auxx3
    GROUP BY genre2) st1
    INNER JOIN auxx3
    ON st1.genre2 = auxx3.genre1 
    WHERE st1.max_metric = auxx3.metric
    ORDER BY st1.genre2, auxx3.metric DESC
""")

# Display
print(f'\n{auxx4.count()} rows in total')
auxx4.show(auxx4.count())


22 rows in total
+---------+-----------------+----------+
|   genre3|         director|max_metric|
+---------+-----------------+----------+
|   Action|Christopher Nolan|     64.40|
|Adventure| Steven Spielberg|     61.28|
|Animation|      Lee Unkrich|     49.70|
|Biography|  Martin Scorsese|     45.04|
|   Comedy|      Lee Unkrich|     49.70|
|    Crime|  Martin Scorsese|     78.69|
|    Drama|Christopher Nolan|     80.29|
|   Family|   Hayao Miyazaki|     37.55|
|  Fantasy|   Frank Darabont|     16.16|
|Film-Noir|       Carol Reed|      9.83|
|  History|       Mel Gibson|     30.54|
|   Horror|     Ridley Scott|     15.95|
|    Music|  Damien Chazelle|     16.37|
|  Musical|       Gene Kelly|      9.25|
|  Musical|    Stanley Donen|      9.25|
|  Mystery| Alfred Hitchcock|     53.57|
|  Romance|     Billy Wilder|     23.10|
|   Sci-Fi|Christopher Nolan|     50.10|
|    Sport| John G. Avildsen|     15.87|
| Thriller| Alfred Hitchcock|     33.92|
|      War|  Stanley Kubrick|     39.88

In [57]:
spark.stop()